# Download replays from Ballchaser website

In [40]:
from typing import Dict, List, Tuple, Union

In [18]:
import datetime
from ballchaser.client import BallChaser

ball_chaser = BallChaser("AWSZ0JiezKKjPVHQFXNQiPImI6YIKobD6eogELef", backoff=True, max_tries=5)

# search and retrieve replay metadata
replays = [
    replay
    for replay in ball_chaser.list_replays(player_name="GarrettG", replay_count=20, replay_date_after=datetime.datetime(2019, 6,1, tzinfo=datetime.timezone.utc))
]

# retrieve replay statistics
replay_stats = [
    ball_chaser.download_replay(replay["id"], 'replays')
    for replay in replays
]

# Create game frames using Carball

Note: you need boxcars-py 0.1.15 which is not available on pypi, so you need to compile it yourself. 

Also, from time to time again Rocket League may change its replay format. If this just stops working, update the cargo crate boxcars, then rebuild boxcars-py (and reinstall from wheel).

In [50]:
import boxcars_py

class BoosterUpdate:
    frame : int
    boost_pad_name : str
    player_object_id : Union[int, None]
    pickup_value : Union[int, None]

def get_booster_updates(replay_path: str) -> List[BoosterUpdate]:
    with open(replay_path, 'rb') as f:
        buf = f.read()
    replay = boxcars_py.parse_replay(buf)

    names : List[str] = replay['names']
    boost_name_ids = [idx for (idx, name) in enumerate(names) if name.startswith('VehiclePickup_Boost_TA_')]

    current_boost_actor_ids = set()
    actor_id_to_name_id = {}
    booster_updates = []

    for frame_idx, frame in enumerate(replay['network_frames']['frames']):
        for new_actor in frame['new_actors']:
            actor_id = new_actor['actor_id']
            name_id = new_actor['name_id']
            if name_id in boost_name_ids:
                current_boost_actor_ids.add(actor_id)
                actor_id_to_name_id[actor_id] = name_id
            else:
                if actor_id in current_boost_actor_ids:
                    current_boost_actor_ids.remove(actor_id)

        for updated_actor in frame['updated_actors']:
            actor_id = updated_actor['actor_id']
            if actor_id in current_boost_actor_ids and 'PickupNew' in updated_actor['attribute']:
                update = BoosterUpdate()
                update.frame = frame_idx
                update.boost_pad_name = names[actor_id_to_name_id[actor_id]]
                update.player_object_id = updated_actor['attribute']['PickupNew']['instigator']
                update.pickup_value = updated_actor['attribute']['PickupNew']['picked_up']
                
                booster_updates.append(update)
    
    return booster_updates

In [77]:
class KeyWrapper:
    def __init__(self, iterable, key):
        self.it = iterable
        self.key = key

    def __getitem__(self, i):
        return self.key(self.it[i])

    def __len__(self):
        return len(self.it)

In [103]:
from rlgym.utils.gamestates import GameState, PhysicsObject, PlayerData
from rlgym.utils.math import euler_to_rotation, rotation_to_quaternion
from pandas import DataFrame, Series
from bisect import bisect_left
from carball.analysis.analysis_manager import AnalysisManager
import numpy as np
import carball
from scipy.spatial import KDTree
from rlgym.utils.common_values import BOOST_LOCATIONS
import pandas as pd

boost_pad_location_tree = KDTree(BOOST_LOCATIONS)

def get_orange_goal_frames(analysis_manager: AnalysisManager) -> List[int]:
    return [g.frame_number for g in analysis_manager.game.goals if g.player_team]
def get_blue_goal_frames(analysis_manager: AnalysisManager) -> List[int]:
    return [g.frame_number for g in analysis_manager.game.goals if not g.player_team]
def read_physics_object(row: Series, name: str) -> PhysicsObject:
    o = PhysicsObject()
    o.position = np.array([row['pos_x'][name], row['pos_y'][name], row['pos_z'][name]])
    o.linear_velocity = np.array([row['vel_x'][name], row['vel_y'][name], row['vel_z'][name]])
    o.angular_velocity = np.array([row['ang_vel_x'][name], row['ang_vel_y'][name], row['ang_vel_z'][name]])
    rotation_euler = np.array([row['rot_x'][name], row['rot_y'][name], row['rot_z'][name]])
    o.quaternion = euler_to_rotation(rotation_euler)
    return o

def quaternion_multiply(quaternion0, quaternion1):
    """ From https://stackoverflow.com/q/40915069
    
    Return multiplication of two quaternions.

    >>> q = quaternion_multiply([1, -2, 3, 4], [-5, 6, 7, 8])
    >>> numpy.allclose(q, [-44, -14, 48, 28])
    True

    """
    x0, y0, z0, w0 = quaternion0
    x1, y1, z1, w1 = quaternion1
    return np.array((
         x1*w0 + y1*z0 - z1*y0 + w1*x0,
        -x1*z0 + y1*w0 + z1*x0 + w1*y0,
         x1*y0 - y1*x0 + z1*w0 + w1*z0,
        -x1*x0 - y1*y0 - z1*z0 + w1*w0), dtype=np.float64)

def read_physics_object_inverted(row: Series, name: str):
    o = PhysicsObject()
    o.pos = np.array([row['pos_x'][name], row['pos_y'][name], row['pos_z'][name]]) * np.array([-1, -1, 1])
    o.linear_velocity = np.array([row['vel_x'][name], row['vel_y'][name], row['vel_z'][name]]) * np.array([-1, -1, 1])
    o.angular_velocity = np.array([row['ang_vel_x'][name], row['ang_vel_y'][name], row['ang_vel_z'][name]]) * np.array([-1, -1, 1])
    rotation_euler = np.array([row['rot_x'][name], row['rot_y'][name], row['rot_z'][name]])
    o.quaternion = quaternion_multiply(rotation_to_quaternion(euler_to_rotation(rotation_euler)), np.array([0, 0, 0, -1]))


BOOST_FULL : float = 1
BOOST_EMPTY : float = 0

def get_game_states(replay_path : str):
    boost_updates = get_booster_updates(replay_path)
    boost_name_to_index : Dict[str, int] = dict()
    analysis_manager = carball.analyze_replay_file(replay_path)
    player_id_to_index = {p.id.id : idx for (idx, p) in enumerate(analysis_manager.protobuf_game.players)}
    players : Tuple[str, bool] = [(p.name, p.is_orange) for p in analysis_manager.game.players]
    hits : List[Tuple[int, int]] = [(hit.frame_number, player_id_to_index[hit.player_id.id]) for hit in  analysis_manager.protobuf_game.game_stats.hits]
    orange_goal_frames = get_orange_goal_frames(analysis_manager)
    blue_goal_frames = get_blue_goal_frames(analysis_manager)
    df = analysis_manager.get_data_frame()

    last_touch = None
    boost_pads : np.ndarray = np.array([BOOST_FULL] * GameState.BOOST_PADS_LENGTH)
    
    for row in df.iterrows():
        frame = int(row[0])
        row = row[1].unstack()

        orange_goals = sum(1 for g in orange_goal_frames if g <= frame)
        blue_goals = sum(1 for g in blue_goal_frames if g <= frame)

        player_objects = []
        player_index = {}
        for name, is_orange in players:
            player = PlayerData()
            player.car_data = read_physics_object(row, name)
            player.inverted_car_data = read_physics_object_inverted(row, name)
            #TODO: player.on_ground
            player.boost_amount = row['boost'][name]
            
            if not pd.isna(row['boost_collect'][name]):
                player_pickups = [p for p in boost_updates if p.player_object_id ] # im dying here. jetzt muss ich die auch noch irgendwo her holen
                bisect_left(KeyWrapper())
                print(f'frame {frame} player {name} picked up boost at {player.car_data.position}')

            player_index[name] = len(player_objects)
            player_objects.append(player)
            

        ball = read_physics_object(row, 'ball')
        touch_index = bisect_left(KeyWrapper(hits, key=lambda c: c[0]), frame)
        last_touch = hits[touch_index - 1][1] if touch_index > 0 else None
        
        state = GameState()
        state.game_type = 0
        state.blue_score = blue_goals
        state.orange_score = orange_goals
        state.ball = ball
        state.last_touch = last_touch
        state.players = player_objects

        state.boost_pads = boost_pads.copy()
        state.inverted_boost_pads = state.boost_pads[::-1]

        yield state

In [102]:
[f.frame for f in get_booster_updates(replays[0])]

[126,
 126,
 162,
 167,
 209,
 239,
 239,
 257,
 281,
 283,
 283,
 283,
 283,
 283,
 283,
 283,
 324,
 404,
 429,
 436,
 443,
 448,
 461,
 488,
 517,
 538,
 542,
 549,
 556,
 562,
 563,
 563,
 563,
 563,
 563,
 563,
 563,
 563,
 563,
 563,
 563,
 563,
 563,
 573,
 591,
 596,
 601,
 614,
 615,
 644,
 670,
 705,
 719,
 728,
 741,
 759,
 761,
 790,
 832,
 837,
 841,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 845,
 854,
 872,
 874,
 878,
 895,
 903,
 925,
 950,
 988,
 994,
 1026,
 1063,
 1085,
 1099,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1125,
 1129,
 1136,
 1141,
 1155,
 1197,
 1242,
 1254,
 1266,
 1274,
 1372,
 1381,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1409,
 1421,


In [100]:
import glob

replays = glob.glob("replays/*")
replay_path = replays[0]

i = 0
for gs in get_game_states(replay_path):
    i += 1
    if i >= 600:
        break
print(i)

Found bot not in bot list
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 3038 by G2 GarrettG
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


frame 128 player CAMA62 picked up boost at [ 1839.51000977 -2408.9699707     42.95000076]
frame 129 player El Ferxxo Mor picked up boost at [-1829.33996582  2364.48999023    52.68000031]
frame 164 player G2 GarrettG picked up boost at [2943.48999023 4145.58984375   19.02000046]
frame 167 player TrickDaddyAlex picked up boost at [-2932.76000977 -3970.31005859    61.15000153]
frame 211 player G2 GarrettG picked up boost at [3621.60009766 2513.98999023   17.        ]
frame 259 player El Ferxxo Mor picked up boost at [3458.05004883  -13.06999969   17.01000023]
frame 283 player CAMA62 picked up boost at [-3475.38989258   -11.92000008    17.01000023]
frame 406 player CAMA62 picked up boost at [  -61.45000076 -2832.9699707     17.01000023]
frame 431 player G2 GarrettG picked up boost at [-3523.55004883 -2441.13989258    77.51999664]
frame 437 player CAMA62 picked up boost at [-1678.06994629 -2211.33007812    17.01000023]
frame 463 player CAMA62 picked up boost at [-2010.79003906 -1105.3900146

In [83]:
df

GarrettG                                                               \
         ping   pos_x   pos_y  pos_z vel_x vel_y vel_z ang_vel_x ang_vel_y   
1          11 -2048.0  2560.0  17.01   0.0   0.0   2.1       0.4       0.2   
2          11 -2048.0  2560.0  17.01   0.0   0.0   2.1       0.4       0.2   
3          11 -2048.0  2560.0  17.01   0.0   0.0   2.1       0.4       0.2   
4          11 -2048.0  2560.0  17.01   0.0   0.0   2.1       0.4       0.2   
5          11 -2048.0  2560.0  17.01   0.0   0.0   2.1       0.4       0.2   
...       ...     ...     ...    ...   ...   ...   ...       ...       ...   
8574       11     NaN     NaN    NaN   NaN   NaN   NaN       NaN       NaN   
8575       11     NaN     NaN    NaN   NaN   NaN   NaN       NaN       NaN   
8576       11     NaN     NaN    NaN   NaN   NaN   NaN       NaN       NaN   
8577       11     NaN     NaN    NaN   NaN   NaN   NaN       NaN       NaN   
8578       11     NaN     NaN    NaN   NaN   NaN   NaN       NaN       NaN   

                ...        game                              \
     ang_vel_z  ...        time     delta seconds_remaining   
1          0.0  ...    4.526104  0.034833               300   
2          0.0  ...    4.559438  0.033334               300   
3          0.0  ...    4.592772  0.033334               300   
4          0.0  ...    4.626105  0.033334               300   
5          0.0  ...    4.659440  0.033336               300   
...        ...  ...         ...       ...               ...   
8574       NaN  ...  311.309998  0.033398                87   
8575       NaN  ...  311.343323  0.033334                87   
8576       NaN  ...  311.376648  0.033335                87   
8577       NaN  ...  311.409973  0.033334                87   
8578       NaN  ...  311.443298  0.033342                87   

                                                                     GarrettG  \
     replicated_seconds_remaining ball_has_been_hit goal_number boost_collect   
1                             NaN              None         NaN           NaN   
2                             NaN              None         NaN           NaN   
3                             NaN              None         NaN           NaN   
4                             NaN              None         NaN           NaN   
5                             NaN              None         NaN           NaN   
...                           ...               ...         ...           ...   
8574                          NaN              True         NaN           NaN   
8575                          NaN              True         NaN           NaN   
8576                          NaN              True         NaN           NaN   
8577                          NaN              True         NaN           NaN   
8578                          NaN              True         NaN           NaN   

           Taroco.         Darou         Forky  
     boost_collect boost_collect boost_collect  
1              NaN           NaN           NaN  
2              NaN           NaN           NaN  
3              NaN           NaN           NaN  
4              NaN           NaN           NaN  
5              NaN           NaN           NaN  
...            ...           ...           ...  
8574           NaN           NaN           NaN  
8575           NaN           NaN           NaN  
8576           NaN           NaN           NaN  
8577           NaN           NaN           NaN  
8578           NaN           NaN           NaN  

[8571 rows x 111 columns]